# Notebook para manutenção do código

In [1]:
import seaborn as sns

## local_lib

In [12]:
#@st.cache
def etl_tesouro_historic_price():
    url = 'https://www.tesourotransparente.gov.br/ckan/dataset/df56aa42-484a-4a59-8184-7676580c81e3/resource/796d2059-14e9-44e3-80c9-2d9e30b405c1/download/PrecoTaxaTesouroDireto.csv'
    df = pd.read_csv(url, sep=';', decimal=',')
    df['data'] = pd.to_datetime(df['Data Base'], format='%d/%m/%Y')
    df['ticker'] = df['Tipo Titulo'].astype(str) + ' ' + df['Data Vencimento'].str[6:]
    df['preco_hist'] = round(df['PU Base Manha'].astype(float), 2)
    return df

#-----------------------------------------------------------------------------------------------------
#@st.cache  
def etl_benchmark_historic_price():
    # CDI
    df_cdi = pd.read_json('http://api.bcb.gov.br/dados/serie/bcdata.sgs.12/dados?formato=json')
    df_cdi['data'] = pd.to_datetime(df_cdi['data'], format='%d/%m/%Y')
    df_cdi.columns = ['data','cdi']

    # IPCA
    df_ipca = pd.read_json('http://api.bcb.gov.br/dados/serie/bcdata.sgs.433/dados?formato=json')
    df_ipca['data'] = pd.to_datetime(df_ipca['data'], format='%d/%m/%Y')
    df_ipca.columns = ['data','ipca']
    df_ipca['ipca'] = round((1 + df_ipca['ipca']) ** (1/22) - 1, 6)

    # IBOV
    df_ibov = yf.download('^BVSP', interval='1d')['Adj Close'].reset_index(drop=False)
    df_ibov.columns = ['data','ibov']
    df_ibov['ibov'] = ((df_ibov['ibov'] / df_ibov['ibov'].shift(1) - 1) * 100).fillna(0).round(6)
        
    # S&P500
    df_sp500 = yf.download('^GSPC', interval='1d')['Adj Close'].reset_index(drop=False)
    df_sp500.columns = ['data','sp500']
    df_sp500['sp500'] = ((df_sp500['sp500'] / df_sp500['sp500'].shift(1) - 1) * 100).fillna(5).round(6)

    df_final = pd.merge(df_ibov, df_sp500, on='data', how='inner')
    df_final = pd.merge(df_final, df_cdi, on='data', how='inner')
    df_final = pd.merge(df_final, df_ipca, on='data', how='left')
    df_final['ipca'] = df_final['ipca'].fillna(method='ffill')
    df_final['data'] = pd.to_datetime(df_final['data'])

    return df_final

## Tesouro Direto

In [19]:
import pandas as pd
import yfinance as yf
import local_lib as lib

### Parte 1: Tabela Dinâmica

In [4]:
# Carregando os dados inputados pelo usuário.
df_tesouro = pd.read_csv('../data/manutencao/dados_pos_home.csv')
df_tesouro['data'] = pd.to_datetime(df_tesouro['data'], format='%Y-%m-%d')
df_tesouro.head()

,data,ticker,qt,preco_mov,vl_total
0,2020-03-20,Tesouro Prefixado 2026,16.05,623.03,9999.63
1,2020-06-04,Tesouro Prefixado 2026,-16.05,708.51,-11371.59
2,2020-04-24,Tesouro Selic 2025,0.94,10585.11,9950.00


In [5]:
# Carregando os preços diários extraídos da API do Tesouro.
df_hist_tesouro = etl_tesouro_historic_price()
df_hist_tesouro.head()

,Tipo Titulo,Data Vencimento,Data Base,Taxa Compra Manha,Taxa Venda Manha,PU Compra Manha,PU Venda Manha,PU Base Manha,data,ticker,preco_hist
0,Tesouro IPCA+ com Juros Semestrais,15/08/2024,09/10/2012,3.70,3.78,2676.99,2658.66,2657.76,2012-10-09,Tesouro IPCA+ com Juros Semestrais 2024,2657.76
1,Tesouro Prefixado,01/01/2013,05/07/2012,7.63,7.66,965.02,964.89,964.61,2012-07-05,Tesouro Prefixado 2013,964.61
2,Tesouro Prefixado,01/01/2014,06/08/2012,7.84,7.88,899.66,899.20,898.92,2012-08-06,Tesouro Prefixado 2014,898.92
3,Tesouro Prefixado,01/01/2015,06/08/2012,8.36,8.42,824.41,823.32,823.05,2012-08-06,Tesouro Prefixado 2015,823.05
4,Tesouro Prefixado com Juros Semestrais,01/01/2014,06/08/2012,7.84,7.88,1036.60,1036.09,1035.77,2012-08-06,Tesouro Prefixado com Juros Semestrais 2014,1035.77


In [6]:
# Juntando os dados do usuário com os dados da API.
df_tesouro_historico = lib.merge_historic_tesouro(df_hist_tesouro, df_tesouro)
df_tesouro_historico.head()

,Tipo Titulo,Data Vencimento,Data Base,Taxa Compra Manha,Taxa Venda Manha,PU Compra Manha,PU Venda Manha,PU Base Manha,data,ticker,preco_hist,qt,preco_mov,vl_total,qt_acum,vl_atualizado
0,Tesouro Prefixado,01/01/2026,20/03/2020,8.04,8.16,640.06,635.97,635.77,2020-03-20,Tesouro Prefixado 2026,635.77,16.05,623.03,9999.63,16.05,10204.11
1,Tesouro Prefixado,01/01/2026,23/03/2020,8.56,8.68,622.77,618.82,618.61,2020-03-23,Tesouro Prefixado 2026,618.61,0.00,0.00,0.00,16.05,9928.69
2,Tesouro Prefixado,01/01/2026,24/03/2020,8.76,8.88,616.40,612.50,612.29,2020-03-24,Tesouro Prefixado 2026,612.29,0.00,0.00,0.00,16.05,9827.25
3,Tesouro Prefixado,01/01/2026,25/03/2020,8.76,8.88,616.61,612.71,612.50,2020-03-25,Tesouro Prefixado 2026,612.50,0.00,0.00,0.00,16.05,9830.62
4,Tesouro Prefixado,01/01/2026,26/03/2020,8.05,8.17,640.50,636.42,636.23,2020-03-26,Tesouro Prefixado 2026,636.23,0.00,0.00,0.00,16.05,10211.49


In [8]:
# Passo extra para criar uma coluna identificando o último dia do mês de cada ativo (útil para os plots)
df_tesouro_historico = lib.create_column_last_day(df_tesouro_historico)
df_tesouro_historico.head()

c:\Users\Kenji\GitHub\b3-report\src\local_lib.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_right['year'] = pd.to_datetime(df_right['data']).dt.year
c:\Users\Kenji\GitHub\b3-report\src\local_lib.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_right['month'] = pd.to_datetime(df_right['data']).dt.month
c:\Users\Kenji\GitHub\b3-report\src\local_lib.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

,Tipo Titulo,Data Vencimento,Data Base,Taxa Compra Manha,Taxa Venda Manha,PU Compra Manha,PU Venda Manha,PU Base Manha,data,ticker,preco_hist,qt,preco_mov,vl_total,qt_acum,vl_atualizado,dummy_ultimo_dia,dt_competencia
0,Tesouro Prefixado,01/01/2026,20/03/2020,8.04,8.16,640.06,635.97,635.77,2020-03-20,Tesouro Prefixado 2026,635.77,16.05,623.03,9999.63,16.05,10204.11,0.0,2020-03-01
1,Tesouro Prefixado,01/01/2026,23/03/2020,8.56,8.68,622.77,618.82,618.61,2020-03-23,Tesouro Prefixado 2026,618.61,0.00,0.00,0.00,16.05,9928.69,0.0,2020-03-01
2,Tesouro Prefixado,01/01/2026,24/03/2020,8.76,8.88,616.40,612.50,612.29,2020-03-24,Tesouro Prefixado 2026,612.29,0.00,0.00,0.00,16.05,9827.25,0.0,2020-03-01
3,Tesouro Prefixado,01/01/2026,25/03/2020,8.76,8.88,616.61,612.71,612.50,2020-03-25,Tesouro Prefixado 2026,612.50,0.00,0.00,0.00,16.05,9830.62,0.0,2020-03-01
4,Tesouro Prefixado,01/01/2026,26/03/2020,8.05,8.17,640.50,636.42,636.23,2020-03-26,Tesouro Prefixado 2026,636.23,0.00,0.00,0.00,16.05,10211.49,0.0,2020-03-01


In [9]:
# Selecionar os tickers a serem visualizados (desnecessário neste notebook)
'''
# Filtro de tickers.
list_ticker = st.multiselect('Escolha o(s) investimento(s):',
                                df_tesouro_historico['ticker'].unique().tolist(),
                                df_tesouro_historico['ticker'].unique().tolist())
df_tesouro_historico = df_tesouro_historico.loc[df_tesouro_historico['ticker'].isin(list_ticker)]
'''

"\n# Filtro de tickers.\nlist_ticker = st.multiselect('Escolha o(s) investimento(s):',\n                                df_tesouro_historico['ticker'].unique().tolist(),\n                                df_tesouro_historico['ticker'].unique().tolist())\ndf_tesouro_historico = df_tesouro_historico.loc[df_tesouro_historico['ticker'].isin(list_ticker)]\n"

In [10]:
# Preparar os dados exclusivamente para a Tabela Dinâmica (visão 1)
df_plot = df_tesouro_historico.loc[df_tesouro_historico['dummy_ultimo_dia'] == 1]
tab1, data_col = lib.custom_pivot_table(df_plot, col_value='vl_atualizado')
tab1.head()

c:\Users\Kenji\GitHub\b3-report\src\local_lib.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dt_competencia'] = pd.to_datetime(df['dt_competencia']).dt.date


,ticker,2020-03-01,2020-04-01,2020-05-01,2020-06-01,2020-07-01,2020-08-01,2020-09-01,2020-10-01,2020-11-01,...,2022-04-01,2022-05-01,2022-06-01,2022-07-01,2022-08-01,2022-09-01,2022-10-01,2022-11-01,2022-12-01,Total
0,Tesouro Prefixado 2026,10666.35,10817.86,11200.65,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,32684.86
1,Tesouro Selic 2025,0.00,9950.87,9972.85,9994.69,10014.52,10030.87,9988.26,9995.85,9999.23,...,10859.45,10978.86,11095.78,11214.44,11351.05,11479.19,11601.8,11720.95,11806.51,346738.97
2,Total,10666.35,20768.73,21173.50,9994.69,10014.52,10030.87,9988.26,9995.85,9999.23,...,10859.45,10978.86,11095.78,11214.44,11351.05,11479.19,11601.8,11720.95,11806.51,379423.83


### Parte 2: Gráfico de Linha com Benchmark

In [17]:
# Extração das variações dos benchmarks (API do BC e yfinance)
df_hist_bench = etl_benchmark_historic_price()
df_hist_bench.head()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,data,ibov,sp500,cdi,ipca
0,1993-04-27,0.000000,1.031047,1.389667,NaN
1,1993-04-28,-0.816330,0.002278,1.374333,NaN
2,1993-04-29,-2.469130,0.198627,1.346333,NaN
3,1993-04-30,1.687762,0.296199,1.302333,NaN
4,1993-05-03,0.000000,0.515684,1.305333,NaN


In [20]:
# Junção dos dados inputados pelo usuário e os dados históricos do benchmarks
df_tesouro_historico_agg = lib.merge_historic_benchmark(df_tesouro_historico, df_hist_bench)
df_tesouro_historico_agg.head()

,data,ibov,sp500,cdi,ipca,vl_total,vl_atualizado
0,2020-03-20,-1.848329,-4.335951,0.014227,0.018909,9999.63,10204.11
1,2020-03-23,-5.217015,-2.929387,0.014227,0.018909,0.00,9928.69
2,2020-03-24,9.688532,9.382774,0.014227,0.018909,0.00,9827.25
3,2020-03-25,7.496164,1.153501,0.014227,0.018909,0.00,9830.62
4,2020-03-26,3.674156,6.241416,0.014227,0.018909,0.00,10211.49


In [21]:
# Arrumando dados em um formato melhor para o gráfico.
tab2 = lib.custom_data_lineplot(df_tesouro_historico_agg, ['ibov', 'sp500', 'cdi', 'ipca'])
tab2.head()

,data,variable,value
0,2020-03-20,Carteira,10204.11
1,2020-03-23,Carteira,9928.69
2,2020-03-24,Carteira,9827.25
3,2020-03-25,Carteira,9830.62
4,2020-03-26,Carteira,10211.49
